In [15]:
%%time
import os
import subprocess
import numpy as np
import pandas as pd
import scipy.io.wavfile
from sys import getsizeof
from scipy import signal
from IPython.display import Audio
import csv
import matplotlib.pyplot as plt
import seaborn as sns

# feature extraction with pyAudioAnalysis:
from pyAudioAnalysis import ShortTermFeatures

import cudf
import cuspatial
import cupy as cp
import cusignal

print(cuspatial.point_in_polygon_bitmap)
print(cudf.__version__)

import warnings
warnings.filterwarnings('ignore')

<function point_in_polygon_bitmap at 0x7f434412a1e0>
0.13.0
CPU times: user 259 µs, sys: 40 µs, total: 299 µs
Wall time: 266 µs


In [16]:
# Let's fix target parameters:
target_length = 5 # seconds
target_sr = 48000 # sampling rate, samples per second

In [17]:
#### ------------------------------------------------------------
####  wav -> resample
#### ------------------------------------------------------------
### define procedure to resample
def resample_wav(input_file):
    sr, wave = scipy.io.wavfile.read(input_file)

    # some wav files can be corrupted, e.g. without any samples with file size 
    # ~0 bytes - 100 bytes which seems to contain only headers but no data
    if len(wave) == 0:
        print(f"\n[Bad file] {input_file}")
        os.remove(input_file)
        return None
    # Let's fix target parameters:
    target_length = 5 # seconds
    target_sr = 48000 # sampling rate, samples per second
    
    # resample on CPU/GPU
    try:
        wave_gpu = cp.asarray(wave)
        resampled_wave_gpu = cusignal.resample(wave_gpu, target_sr*target_length)
        resampled_wave = cp.asnumpy(resampled_wave_gpu).astype(int)
        # resampled_wave_cpu = scipy.signal.resample(wave, int(target_sr*target_length))
        # resampled_wave = resampled_wave_cpu.astype(int)
        scaled_wave = np.int16(resampled_wave/np.max(np.abs(resampled_wave)) * 32767)
    except:
        scipy.io.wavfile.write("./tmp.wav", sr, wave)
        os.chown("./tmp.wav", 1000, 1000)
        print('File:', input_file)
        print('Data:', wave)
        print('Sampling rate:', target_sr)
        print('Original samples:', len(wave))
        print('Original audio length:', round(wave.size/sr, 4), 'seconds')
        print('Lowest amplitude:', min(wave))
        print('Highest amplitude:', max(wave))
        print('Read retries:', read_retry)
        Audio("./tmp.wav")
        raise SystemExit("Stopping further execution on error.")
    return scaled_wave

In [18]:
%%time 
source_dir = "../data/sliced/"
source_files = []
target_dir = "../data/dataset/resampled/"
target_sr = 48000 # sampling rate, samples per second


# sort all files
for j in sorted(os.listdir(source_dir)):
    source_files += [f"{source_dir}{j}"]
print(f"{len(source_files)} files sorted, resampling")
print()

# need to handle memory management manually
mempool = cp.get_default_memory_pool()
pinned_mempool = cp.get_default_pinned_memory_pool()

# generate featupes per each file and put to dataframe
for idx, one_file in enumerate(source_files):
    resampled_wav = resample_wav(one_file)
    target_file = os.path.join(target_dir, os.path.basename(one_file))
    scipy.io.wavfile.write(target_file, target_sr, resampled_wav)
    os.chown(target_file, 1000, 1000)
    if idx % 5 == 0:
        # nice feature, printing on the same line :)
        print(f"\r[{idx}/{len(source_files)}] {round(100.0 * idx/len(source_files), 2)}%: {one_file}", end = "")
        # You can clear the memory pool by calling `free_all_blocks`.
        mempool.free_all_blocks()
        pinned_mempool.free_all_blocks()

78581 files sorted, resampling

[78580/78581] 100.0%: ../data/sliced/9_96_5.04_8855.wavCPU times: user 7min 9s, sys: 3min 23s, total: 10min 32s
Wall time: 13min 42s
